<a href="https://colab.research.google.com/github/reynoso811/Image-Classification-Pneumonia-X-rays-/blob/main/Phase_04_Talos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd 
import random as rn

# tensorflow
import tensorflow.random as tfr
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Lambda, Input
from tensorflow.keras.layers import Conv2D, MaxPool2D, MaxPooling2D, BatchNormalization
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Plotting
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import seaborn as sns

from skimage import color, exposure
from sklearn.metrics import classification_report

import os
import cv2

# Setting the same seed for repeatability

seed = 0

np.random.seed(seed) 
rn.seed(seed)
tfr.set_seed(seed)

# Display graphs in a Jupyter
%matplotlib inline

print("Imported")

Imported


In [ ]:
# Google Drive connection

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create a path to the root folder and path to specific folders

data_path = '/content/drive/MyDrive/Phase_04_project/chest_xray/chest_xray/'
data_path

train_path = data_path + 'train/'
test_path = data_path + 'test/'
val_path = data_path + 'val/'

In [ ]:
# Optimal image size for modeling

img_size = 200

In [ ]:
# Custom function to read data from folders
# Returns numpy array [img, label]

def read_data(data_paths):
    for data_path in data_paths:
        labels = ['PNEUMONIA', 'NORMAL']
        images = []
        y = []
        for label in labels:
            curr_path = data_path + label
            for img in os.listdir(curr_path):
                if ('DS' not in img):
                    image_path = os.path.join(curr_path, img)
                    image =  cv2.resize(cv2.imread(image_path), (img_size, img_size))
                    if image is not None:
                        images.append([image, label])
                
    images = np.asarray(images)
    return images

In [ ]:
%%time
train = read_data([train_path])
test = read_data([val_path, test_path])

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


CPU times: user 1min 28s, sys: 4.54 s, total: 1min 33s
Wall time: 1min 52s


In [ ]:
# Shuffle the data

for i in range(10):
    np.random.shuffle(train)
    np.random.shuffle(test)

In [ ]:
# Create data frame for visualization

train_df = pd.DataFrame(train, columns=['image', 'label'])
test_df = pd.DataFrame(test, columns = ['image', 'label'])

In [ ]:
def lung_condition(label):
    if label == 'NORMAL':
        return 0
    else:
        return 1

In [ ]:
# Function for dividing data into arrays X and y

def splitdata(data):
    X = []
    y = []
    for i, (val, label) in enumerate(data):
        X.append(val)
        y.append(lung_condition(label))
    return np.array(X), np.array(y)

In [ ]:
# Split the data

np.random.shuffle(train)
np.random.shuffle(test)
X_train, y_train = splitdata(train)
X_test, y_test = splitdata(test)


In [ ]:
# Function for drawing learning curve history on learning neural network

def draw_learning_curve(history, keys=['accuracy', 'loss']):
    plt.figure(figsize=(20,8))
    for i, key in enumerate(keys):
        plt.subplot(1, 2, i + 1)
        sns.lineplot(x = history.epoch, y = history.history[key])
        sns.lineplot(x = history.epoch, y = history.history['val_' + key])
        plt.title('Learning Curve')
        plt.ylabel(key.title())
        plt.xlabel('Epoch')
#         plt.ylim(ylim)
        plt.legend(['train', 'test'], loc='best')
    plt.show()

In [ ]:
# Lee code

In [ ]:
#create function that helps us visualize our model
def visualize_training_results(results):
    history = results.history
    plt.figure()
    plt.plot(history['val_loss'])
    plt.plot(history['loss'])
    plt.legend(['val_loss', 'loss'])
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()
    
    plt.figure()
    plt.plot(history['val_accuracy'])
    plt.plot(history['accuracy'])
    plt.legend(['val_acc_accuracy', 'accuracy'])
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()

In [ ]:
#set image dimensions
IMG_HEIGHT = 224
IMG_WIDTH = 224
batch_size = 624

In [ ]:
image_gen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_data_gen = image_gen.flow_from_directory(
    directory=train_path,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size= 5216,
    class_mode='binary')

Found 5216 images belonging to 2 classes.


In [ ]:
test_data_gen = image_gen.flow_from_directory(
    directory = test_path,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = batch_size,
    class_mode='binary')

Found 624 images belonging to 2 classes.


In [ ]:
#creating X and y test, train datasets
X_train, y_train = next(train_data_gen)
X_test, y_test = next(test_data_gen)

In [ ]:
#in order to tune our CNN, we need to figure out what parameters work the best 
#Below are the parameters I've chosen:
params = {'dropout': [0.1, 0.3, 0.5],
          'optimizer': ['adam', 'sgd'],
          'activation1': ['relu', 'tanh']}

In [ ]:
#use generic CNN model with parameters options.
def tuned_model(X_train, y_train, X_test, y_test, params):
    model = Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation=params['activation1'], input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), padding='same'))
    model.add(layers.MaxPool2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation=params['activation1'], padding='same'))
    model.add(layers.MaxPool2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation=params['activation1'], padding='same'))
    model.add(layers.MaxPool2D(pool_size=(3, 3)))
    model.add(layers.Flatten())
    model.add(layers.Dense(120, activation=params['activation1']))
    model.add(layers.Dense(60, activation=params['activation1']))
    model.add(layers.Dropout(params['dropout']))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=params['optimizer'], metrics='accuracy')
    es = EarlyStopping(patience=20, monitor= 'val_accuracy', restore_best_weights=True)

    out = model.fit(X_train, 
                y_train,
                batch_size = 50,
                epochs = 30,
                validation_data=(X_test, y_test),
                callbacks = es)

    return out, model

In [ ]:
!pip install talos

In [ ]:
import talos

In [ ]:
%%time
results = talos.Scan(X_train, y_train, params=params, model=tuned_model, experiment_name='grid')

  0%|          | 0/12 [00:00<?, ?it/s]

Epoch 1/30
74/74 [==============================] - 386s 5s/step - loss: 0.3177 - accuracy: 0.8666 - val_loss: 0.1986 - val_accuracy: 0.9137
Epoch 2/30
74/74 [==============================] - 384s 5s/step - loss: 0.1251 - accuracy: 0.9521 - val_loss: 0.1073 - val_accuracy: 0.9565
Epoch 3/30
74/74 [==============================] - 381s 5s/step - loss: 0.1823 - accuracy: 0.9417 - val_loss: 0.1446 - val_accuracy: 0.9425
Epoch 4/30
74/74 [==============================] - 379s 5s/step - loss: 0.1629 - accuracy: 0.9359 - val_loss: 0.0997 - val_accuracy: 0.9668
Epoch 5/30
74/74 [==============================] - 378s 5s/step - loss: 0.1014 - accuracy: 0.9627 - val_loss: 0.0812 - val_accuracy: 0.9661
Epoch 6/30
74/74 [==============================] - 380s 5s/step - loss: 0.0680 - accuracy: 0.9753 - val_loss: 0.0798 - val_accuracy: 0.9700
Epoch 7/30
74/74 [==============================] - 384s 5s/step - loss: 0.0563 - accuracy: 0.9784 - val_loss: 0.0865 - val_accuracy: 0.9719
Epoch 8/30
74

  8%|▊         | 1/12 [3:12:24<35:16:33, 11544.83s/it]

Epoch 1/30
74/74 [==============================] - 385s 5s/step - loss: 0.5720 - accuracy: 0.7371 - val_loss: 0.7470 - val_accuracy: 0.2550
Epoch 2/30
74/74 [==============================] - 386s 5s/step - loss: 0.5554 - accuracy: 0.7286 - val_loss: 0.8689 - val_accuracy: 0.2550
Epoch 3/30
74/74 [==============================] - 386s 5s/step - loss: 0.5734 - accuracy: 0.7165 - val_loss: 0.8758 - val_accuracy: 0.2550
Epoch 4/30
74/74 [==============================] - 387s 5s/step - loss: 0.5140 - accuracy: 0.7423 - val_loss: 0.7313 - val_accuracy: 0.7450
Epoch 5/30
74/74 [==============================] - 386s 5s/step - loss: 0.4524 - accuracy: 0.8003 - val_loss: 1.6852 - val_accuracy: 0.2550
Epoch 6/30
74/74 [==============================] - 386s 5s/step - loss: 0.4334 - accuracy: 0.8102 - val_loss: 0.6882 - val_accuracy: 0.7450
Epoch 7/30
74/74 [==============================] - 386s 5s/step - loss: 0.3331 - accuracy: 0.8762 - val_loss: 0.2922 - val_accuracy: 0.8709
Epoch 8/30
74

In [ ]:
results.best_model(metric='val_accuracy')